# PCO. Création de la BDD relationnelle pour l'application
## Compétences C3. C4. C10. C11. 

![IllustrationDatabase2](Ressources_NB\IllustrationDatabase2.png)

Ce fichier présente l'ensemble des scripts de création de BDD, de table, de requêtes d'import/export des données retenues pour la démonstration de l'application. L'ensemble de ces données forme le Dataset **Test** construit en amont lors du développement des modèles IA. De plus, ces données sont prétraitées en amont et directement importées en BDD comme mentionné dans le Notebook *Création d'une BDD relationnelle démonstrative*. Pour valider la compétence C11, 2 autres BDD non-relationnelle sont créées à l'occasion pour mettre à jour les données (Changement de Mot de passe) ou insérer de nouvelles (Historique de performances IA).

## Compétences
* **C3.** Programmer l’import de données initiales nécessaires au projet en base de données, afin de les rendre exploitables par un tiers, dans un langage de programmation adapté et à partir de la stratégie de nettoyage des données préalablement définie.
* **C4.** Préparer les données disponibles depuis la base de données analytique en vue de leur utilisation par les algorithmes d’intelligence artificielle.
* **C10.** Concevoir une base de données relationnelle à l’aide de méthodes standards de modélisation de données.
* **C11.** Développer les requêtes et les composants d'accès aux données dans un langage adapté afin de persister et mettre à jour les données issues de l’application en base de données.

## Librairies

In [1]:
# Librairies standards de data-analyses:
    
import numpy as np
from numpy import set_printoptions
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
from scipy.stats import norm, skew
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ttest_ind


# sklearn modules Preprocessing:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# sklearn modules Model Selection:

from sklearn import tree, linear_model, neighbors
from sklearn import naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

import xgboost
from xgboost import XGBClassifier


# sklearn modules Model Evaluation & Improvement:
    
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score
from sklearn.metrics import make_scorer, recall_score, log_loss, matthews_corrcoef
from sklearn.metrics import average_precision_score
from sklearn.pipeline import make_pipeline


# Librairies standards de data-visualisation:

import seaborn as sn
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib 
%matplotlib inline
color = sn.color_palette()
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
from IPython.display import display
pd.options.display.max_columns = None
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date, datetime
import re


# Importation IA:
import joblib


# Filtre warnings:

import warnings 
warnings.filterwarnings("ignore")


# Coloration des graphes Seaborn:

colors = ['#440154', '#482475','#414487','#355F8D','#2A788E','#21918C','#22A884','#44BF70','#7AD151','#BDDF26','#FDE725']
red = '#fd8181'
blue = '#8198fd'

## Fonctions

In [2]:
# Fonction détection des valeurs manquantes:
def find_NaN(dataset):
    NaN = dataset.isnull().sum().to_frame('Valeurs Manquantes')
    NaN = NaN.loc[NaN['Valeurs Manquantes']>0]
    NaN['Pourcentage'] = [str(round((i*100)/len(dataset),1)) for i in NaN['Valeurs Manquantes']]
    NaN['Pourcentage'] = [i+' %' for i in NaN['Pourcentage']]
    if NaN.empty:
        return print(f'Aucune valeur manquante n\'est détectée sur ce dataset')
    else:
        return NaN
    
# Fonction transformation date int(YYYYMMDD) en str(YYYY-MM-DD):    
def Transform_date(serie):
    Liste = []
    for i in serie:
        i = str(i)
        y = i[:4]
        m = i[4:6]
        d = i[6:]
        i = f'{d}-{m}-{y}'
        Liste.append(i)

    return Liste

# Fonction transformation date str(YYYY-MM-DD) en int(YYYYMMDD):    
def Format_intdate(serie):
    Liste = []
    for i in serie:
        #i = i.strftime("%Y-%m-%d")
        y = i[:4]
        m = i[5:7]
        d = i[8:]
        i = f"{y}{m}{d}"
        i = int(i)
        Liste.append(i)

    return Liste

def Transform_date_MonthYear(serie):
    Liste = []
    for i in serie:
        i = str(i)
        y = i[:4]
        m = i[4:6]
        i = f'{y}{m}'
        Liste.append(i)

    return Liste

# Différence entre 2 dates en jour:
def difference_dates(date1, date2):
    return abs(date2-date1).days

# Transformation datetime:
def TransformationToDate(serie):
    new_serie = [datetime.strptime(i, "%d-%m-%Y") for i in serie]
    return new_serie

**Schéma formation des données importées en BDD relationnelle :**
![schema_7](Ressources_NB\schema_7.png)

## Création de la BDD C10.

## Insertion des données dans la base relationnelle C3.

In [3]:
Test = pd.read_csv('Test_db.csv')
Test

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_count,city,bd,gender,registered_via,registration_init_time,num_25Sum,num_50Sum,num_75Sum,num_985Sum,num_100Sum,num_unqSum,total_secsSum,num_25Mean,num_50Mean,num_75Mean,num_985Mean,num_100Mean,num_unqMean,total_secsMean,count,is_churn,price_per_day,days_fidelity
0,RGJpGXR8B36y,41,30,99,99,1,2017-03-26,2017-04-26,0,1,1,-1,inconnu,7,2016-06-27,72.0,18.0,14.0,9.0,139.0,130.0,44905.076,6.000000,1.500000,1.166667,0.750000,11.583333,10.833333,3742.089667,12,0,3.300000,303
1,IsgqEe3yJvkE,41,30,129,129,1,2017-03-31,2017-05-04,0,2,5,37,male,7,2014-03-18,85.0,19.0,12.0,19.0,179.0,219.0,93092.186,3.148148,0.703704,0.444444,0.703704,6.629630,8.111111,3447.858741,27,0,4.300000,1143
2,7TP4Su5pN54P,41,30,129,129,1,2017-03-07,2017-04-08,0,1,1,-1,inconnu,7,2011-04-08,135.0,40.0,32.0,41.0,302.0,463.0,90659.922,4.655172,1.379310,1.103448,1.413793,10.413793,15.965517,3126.204207,29,0,4.300000,2192
3,m/y68t7nHIrt,29,30,180,180,1,2017-03-13,2017-04-13,0,1,1,-1,inconnu,3,2012-10-31,27.0,1.0,0.0,4.0,211.0,226.0,52312.882,1.038462,0.038462,0.000000,0.153846,8.115385,8.692308,2012.033923,26,0,6.000000,1625
4,seOJZ4R/Yq/N,41,30,149,149,1,2017-03-05,2017-04-06,0,1,1,-1,inconnu,7,2011-11-26,222.0,49.0,33.0,43.0,731.0,823.0,184460.900,8.880000,1.960000,1.320000,1.720000,29.240000,32.920000,7378.436000,25,0,4.966667,1958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43539,0aLSP2IKkqd1,41,30,99,99,1,2017-03-10,2017-04-10,0,1,1,-1,inconnu,7,2012-04-13,11.0,4.0,3.0,3.0,77.0,60.0,20044.835,1.222222,0.444444,0.333333,0.333333,8.555556,6.666667,2227.203889,9,0,3.300000,1823
43540,jGus6KC5YfOe,33,30,149,149,1,2017-03-31,2017-05-02,0,2,5,27,female,9,2010-05-13,0.0,0.0,0.0,1.0,2.0,3.0,699.825,0.000000,0.000000,0.000000,1.000000,2.000000,3.000000,699.825000,1,0,4.966667,2546
43541,HBrK+xg29CvY,41,30,99,99,1,2017-03-05,2017-04-05,0,1,1,-1,inconnu,7,2016-07-06,2.0,2.0,0.0,0.0,0.0,3.0,286.824,2.000000,2.000000,0.000000,0.000000,0.000000,3.000000,286.824000,1,0,3.300000,273
43542,LVrjliPLeJO8,36,30,180,180,1,2017-03-03,2017-04-02,0,1,13,-1,inconnu,9,2011-04-26,35.0,7.0,6.0,5.0,29.0,77.0,11395.002,4.375000,0.875000,0.750000,0.625000,3.625000,9.625000,1424.375250,8,0,6.000000,2168


In [ ]:
# Ajout des variables mois et années sur les dates d'expiration d'abonnement en vue de futures requêtes dans l'application:
Test['month_expire'] = [int(i[5:7]) for i in Test['membership_expire_date']]
Test['year_expire'] = [int(i[:4]) for i in Test['membership_expire_date']]

# Conversion des variables dates en vu de leur intégration en base SQL:
intermate_date2 = []
for i in Test['transaction_date']:
    s = str(i)
    date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])).strftime('%Y:%m:%d')
    intermate_date2.append(date)

Test['transaction_date'] = intermate_date2

intermate_date3 = []
for i in Test['membership_expire_date']:
    s = str(i)
    date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])).strftime('%Y:%m:%d')
    intermate_date3.append(date)

Test['membership_expire_date'] = intermate_date3

intermate_date4 = []
for i in Test['registration_init_time']:
    s = str(i)
    date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8])).strftime('%Y:%m:%d')
    intermate_date4.append(date)

Test['registration_init_time'] = intermate_date4

In [ ]:
# Encodage msno:
le = LabelEncoder()
le.fit(Test['msno'].tolist())
Test['ID_user'] = le.transform(Test['msno'].tolist())
Test['ID_user'] = [i+1 for i in Test['ID_user']]

**Structure de la base de données relationnelles :**
![schema_6](Ressources_NB\schema_6.png)

## Démonstration sur la base de données connectée à l'IA C4.
Cette BDD relationnelle permet la récupération des données d'utilisateurs prétraitées en fonction d'une date spécifique en sélectionnant une année et un mois d'expiration de l'abonnement. La page générée sur l'application renvoie un tableau repertoriant les utilisateurs sélectionnés et le score de risque calculé par l'IA.

![screen_app2](Ressources_NB\screen_app2.png)
![screen_app1](Ressources_NB\screen_app1.png)

In [56]:
# Importation des Fichiers de Modèles et Scaler:
scaler = joblib.load('ProcessModel/ScalerXGBC_BF.joblib')
colonnesmodel = joblib.load('ProcessModel/ColonnesXGBC_BF.joblib')
classifier = joblib.load('ProcessModel/XGBC_BF.joblib')
colonnescaler = joblib.load('ProcessModel/ColonnesForScale.joblib')

In [51]:
# Sélection de la date d'expiration, ici Avril 2017:
year = 2017
month = 4

### Requêtes

In [52]:
import mysql.connector
config = {
      'user': 'root',
      'password': 'root',
      'host': '127.0.0.1',
      'port': '3307',
      'database': 'databasekkbox',
      'raise_on_warnings': True,
    }

# Création du Cursor
link = mysql.connector.connect(**config)
cursor = link.cursor(buffered=True)

# Requêtes Transactions avec variables d'expiration d'abonnement reliés à l'identifiant msno via la jointure ID_user:
query = """
            SELECT 
                Transactions.payment_method_id, 
                Transactions.payment_plan_days,
                Transactions.plan_list_price,
                Transactions.actual_amount_paid,
                Transactions.is_auto_renew,
                Transactions.transaction_date,
                Transactions.membership_expire_date,
                Transactions.is_cancel,
                Transactions.transaction_count,
                Transactions.price_per_day,
                User.msno 
            FROM transactions 
            JOIN User on Transactions.User_ID = User.User_ID 
            WHERE Transactions.year_expire = %s 
            AND Transactions.month_expire = %s
        """
cursor.execute(query, (int(year), int(month)))
rows_transaction = cursor.fetchall()

Transactions_sql = []
for values in rows_transaction :
    Transactions_sql.append(list(values))

# Requêtes Logs reliés à l'identifiant msno via la jointure ID_user:
query = """
            SELECT 
                Logs.num_25Sum, 
                Logs.num_50Sum, 
                Logs.num_75Sum, 
                Logs.num_985Sum, 
                Logs.num_100Sum, 
                Logs.num_unqSum, 
                Logs.total_secsSum, 
                Logs.num_25Mean, 
                Logs.num_50Mean,
                Logs.num_75Mean, 
                Logs.num_985Mean, 
                Logs.num_100Mean, 
                Logs.num_unqMean, 
                Logs.total_secsMean,
                Logs.count, 
                User.msno 
            FROM Logs 
            JOIN User on Logs.User_ID = User.User_ID
        """ 

cursor.execute(query)
rows_logs = cursor.fetchall()

Logs_sql = []
for values in rows_logs :
    Logs_sql.append(list(values))

# Requêtes Members reliés à l'identifiant msno via la jointure ID_user:
query = """
            SELECT 
                Members.city, 
                Members.bd, 
                Members.gender, 
                Members.registered_via, 
                Members.registration_init_time, 
                Members.days_fidelity,
                User.msno 
            FROM Members 
            JOIN User on Members.User_ID = User.User_ID
        """
cursor.execute(query)
rows_members = cursor.fetchall()

Members_sql = []
for values in rows_members :
    Members_sql.append(list(values))

link.close()

### Transformation des données appelées depuis la BDD en Dataframes

In [55]:
# Transformation en Transactions_sql DataFrame:
colonnes_Transactions_sql = ['payment_method_id', 'payment_plan_days', 'plan_list_price', 'actual_amount_paid', 
                             'is_auto_renew', 'transaction_date', 'membership_expire_date', 'is_cancel', 'transaction_count', 
                             'price_per_day', 'msno']

Transactions_sql = pd.DataFrame(Transactions_sql, columns = colonnes_Transactions_sql)

# Transformation en Logs_sql DataFrame:
colonnes_Logs_sql = ['num_25Sum', 'num_50Sum', 'num_75Sum', 'num_985Sum', 'num_100Sum','num_unqSum', 'total_secsSum', 
                     'num_25Mean', 'num_50Mean', 'num_75Mean', 'num_985Mean', 'num_100Mean', 'num_unqMean', 'total_secsMean', 
                     'count', 'msno']

Logs_sql = pd.DataFrame(Logs_sql, columns = colonnes_Logs_sql)

# Transformation en Members_sql DataFrame:
colonnes_Members_sql = ['city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'days_fidelity', 'msno']
Members_sql = pd.DataFrame(Members_sql, columns = colonnes_Members_sql)

# Merge:
Main = pd.merge(Transactions_sql, Logs_sql, on='msno', how='inner')
Main = pd.merge(Main, Members_sql, on='msno', how='inner')

bd_list = []
for i in Main['bd']:
    if i == 'inconnu':
        bd_list.append(-1)
    else:
        bd_list.append(int(i))
Main['bd'] = bd_list

df = Main.copy()

### Traitement des Dataframes pour les passer au modèle IA

In [57]:
# Modification de series catégorielles:
df['city'] = [str(i)+'C' for i in df.city]
df['payment_method_id'] = [str(i)+'P' for i in df.payment_method_id]
df['registered_via'] = [str(i)+'R' for i in df.registered_via]

# Encodage:
df = pd.concat([df, pd.get_dummies(df.gender)],1)
df = pd.concat([df, pd.get_dummies(df.payment_method_id)],1)
df = pd.concat([df, pd.get_dummies(df.registered_via)],1)
df = pd.concat([df, pd.get_dummies(df.city)],1)
df = df.drop(['gender', 'payment_method_id','registered_via', 'city', 'transaction_date'], 1)


# Calibration avec le dataset de départ ayant servi à la modélisation: 
for feature in colonnescaler:
        if feature not in df.columns:
            df[feature]=0
            
df = df[colonnescaler]

# Suppression des variables temporelles:
df['membership_expire_date'] = Format_intdate(df['membership_expire_date'])
df['registration_init_time'] = Format_intdate(df['registration_init_time'])

# Vérification:
find_NaN(df)

Aucune valeur manquante n'est détectée sur ce dataset


### Remise à l'échelle des données avec le fichier Scaler

In [58]:
X_test = []
X_test = df.drop(['msno','is_churn'], axis = 1)
y_test = df['is_churn']

# Standardisation des données Test avec le scaler fit sur données d'entrainement pour l'intégration Web
X_test2 = pd.DataFrame(scaler.transform(X_test.values))
X_test2.columns = X_test.columns.values
X_test2.index = X_test.index.values
X_test = X_test2

X_test = X_test[colonnesmodel]

y_pred = classifier.predict(X_test)
probability = classifier.predict_proba(X_test)
probability = probability[:,1]

final_results = df
final_results['predictions'] = y_pred
final_results['membership_expire_date'] = df['membership_expire_date']
final_results["propensity_to_churn(%)"] = probability
final_results["propensity_to_churn(%)"] = final_results["propensity_to_churn(%)"]*100
final_results["propensity_to_churn(%)"] = final_results["propensity_to_churn(%)"].astype(int)
final_results = final_results.sort_values(by=['propensity_to_churn(%)'], ascending = False)
Results = final_results[['msno', 'membership_expire_date', 'propensity_to_churn(%)']]
Results.index.msno=None

### Résultats
Application sommaire du modèle IA sur le Dataset construit à partir de cette BDD relationnelle.

In [59]:
Results.head(10)

,msno,membership_expire_date,propensity_to_churn(%)
25852,Ek03E5PQhVJe,20170418,99
12500,asJBI3I9sGY6,20170425,99
33781,Bdl0xsKxugeF,20170418,98
24787,YvfyIgofdSrd,20170430,98
4004,h+WwQghfOZPr,20170415,98
10197,pR12PQ6dqQ6X,20170430,98
27245,PsE2P4E5JF9C,20170414,98
34841,jMmtRCAaPBTw,20170416,98
35210,R5D4rMPUV+5V,20170428,98
13275,sjvaytEYjvjV,20170424,98


In [60]:
# Echantillon des prédictions obtenues:
Table = pd.DataFrame(columns = Results.columns)
for i in range (10, 110, 10):
    j = i - 10
    Filter = Results.loc[Results['propensity_to_churn(%)'] < i]
    Sample = Filter.loc[Filter['propensity_to_churn(%)'] > j].sample(n = 3)
    Table = Table.append(Sample)
    
Table = Table.sort_values(by = ['propensity_to_churn(%)'], ascending = [False])
Table.style.background_gradient(cmap="Blues", subset=['propensity_to_churn(%)'])

,msno,membership_expire_date,propensity_to_churn(%)
21793,vD8lxPe7PJeg,20170403,97
7815,jeDeoQ30/3sj,20170404,95
34612,pBYTvH5BBPPZ,20170401,92
23935,M2cEruZfBjrs,20170417,88
7460,meYQt618y5qW,20170427,86
6418,uF7mYLP6xBtw,20170406,81
29714,ufdsaZ2WLTfK,20170416,76
23369,0y69chLU5D4b,20170424,73
14303,fVlkYixHk3Sr,20170402,71
28742,xw2CUoHu8XT8,20170412,68


## Mise à jour de données dans la BDD C11.
### Base de données sur les utilisateurs de l'application
L'application ne prévoit pas la mise à jour des données dans la BDD principale. Ainsi, une autre BDD, nommé "*Authenticator*", regroupant les informations des utilisateurs se connectant à l'application a été créée :
* Une colonne ID utilisateur Log_ID
* Le nom d'identifiant de l'utilisateur
* Le mot de passe de l'utilisateur

**L'application propose la modification du mot de passe si oublié :**

![screen_app3](Ressources_NB\screen_app3.png)




### Base de données sur les performances du modèle (insertion de données)
Pour suivre les performances du modèle IA sur de nouvelles données, une page de l'application indique les scores, la date de l'évaluation ainsi qu'un graphe de l'historique des performances. Les données de ce graphe viennent de la BDD conçue à cet effet. La page générée envoie les scores en BDD et retourne un graphe mis à jour avec les dernières données.

**Base de données:**
![screen_app5](Ressources_NB\screen_app5.png)

**Graphe généré:**
![screen_app4](Ressources_NB\screen_app4.png)